# Simulating Fluid Flow using Neural Networks

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

## Implementing Basic CNN

In [2]:
def basic_cnn(input_shape, out_c, n_filters=8):
    input = keras.Input(input_shape)
      
    x = layers.Conv2D(n_filters, (3, 3), padding='same', activation='relu')(input)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(padding='same')(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(padding='same')(x)
    
    x = layers.Conv2D(n_filters * 4, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters * 4, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2DTranspose(n_filters * 2, 2, 2, padding='same')(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2DTranspose(n_filters, 2, 2, padding='same')(x)
    
    output = layers.Conv2D(out_c, (1, 1), padding='same')(x)
    
    name = 'basic_cnn{n_filters}'.format(n_filters=n_filters)
    model = keras.Model(input, output, name=name)
    
    return model

In [3]:
basic_model = basic_cnn((64, 64, 4), 3, 8)
basic_model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "basic_cnn8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 4)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 8)         296       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 8)        32        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 8)         584       
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 8)        32        
 hNormalization)                                                 
                                         

2022-04-16 19:17:06.232711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-16 19:17:06.232895: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Implementing UNet

In [4]:
def unet_conv_block(x, n_filters):
    x = layers.Conv2D(n_filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    return x

In [5]:
def unet_down(x, n_filters):
    x = unet_conv_block(x, n_filters)
    skip = layers.MaxPooling2D(padding='same')(x)
    return x, skip

In [6]:
def unet_up(x, skip, n_filters):
    x = layers.Conv2DTranspose(n_filters, 2, 2, padding='same')(x)
    x = layers.Concatenate()([x, skip])
    x = unet_conv_block(x, n_filters)
    return x

In [7]:
def unet(input_shape, out_c, n_filters=8):
    input = keras.Input(input_shape)
    
    # downsampling
    d1, p1 = unet_down(input, n_filters)
    d2, p2 = unet_down(p1, n_filters*2)
    d3, p3 = unet_down(p2, n_filters*4)
    d4, p4 = unet_down(p3, n_filters*8)
    
    # bottleneck
    b = unet_conv_block(p4, n_filters*16)
    
    # upsampling
    u1 = unet_up(b, d4, n_filters*8)
    u2 = unet_up(u1, d3, n_filters*4)
    u3 = unet_up(u2, d2, n_filters*2)
    u4 = unet_up(u3, d1, n_filters)
    
    output = layers.Conv2D(out_c, (1, 1), padding='same')(u4)
    
    name = 'unet{n_filters}'.format(n_filters=n_filters)
    model = keras.Model(input, output, name=name)
    
    return model
    

## Loading Data

In [8]:
fpath = 'data/sim_np/size64/sim_512x64x64x64x3.npy'
dataset = np.load(fpath)
fpath = 'data/sim_np/size64/bound_64x64.npy'
boundary = np.load(fpath)

# Swap the axes representing the number of frames and number of data samples.
# dataset = np.swapaxes(dataset, 0, 1)
# We'll pick out 1000 of the 10000 total examples and use those.
# dataset = dataset[:1000, ...]
# Add a channel dimension since the images are grayscale.
# dataset = np.expand_dims(dataset, axis=-1)

# # Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.9 * dataset.shape[0])]
val_index = indexes[int(0.9 * dataset.shape[0]) :]
train_dataset = dataset[train_index]
val_dataset = dataset[val_index]

# Normalize the data to the 0-1 range.
# train_dataset = train_dataset / 255
# val_dataset = val_dataset / 255

# We'll define a helper function to shift the frames, where
# `x` is frames 0 to n - 1, and `y` is frames 1 to n.
def create_shifted_frames(data, boundary):
    x = np.zeros((data.shape[0], data.shape[1] - 1, data.shape[2], data.shape[3], data.shape[4] + 1), np.float16)
    y = np.zeros((data.shape[0], data.shape[1] - 1, data.shape[2], data.shape[3], data.shape[4]), np.float16)
    
    for i in range(data.shape[0]):
        for j in range(data.shape[1] - 1):
            
            x[i, j] = np.concatenate((data[i, j], np.expand_dims(boundary, axis=-1)), axis=-1)
            y[i, j] = data[i, j + 1]
        
    return x, y


# Apply the processing function to the datasets.
x_train, y_train = create_shifted_frames(train_dataset, boundary)
x_val, y_val = create_shifted_frames(val_dataset, boundary)

# Inspect the dataset.
print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))

Training Dataset Shapes: (460, 63, 64, 64, 4), (460, 63, 64, 64, 3)
Validation Dataset Shapes: (52, 63, 64, 64, 4), (52, 63, 64, 64, 3)


## Training

In [9]:
def unroll_frames(x):
    return x.reshape(x.shape[0]*x.shape[1], x.shape[2], x.shape[3], x.shape[4])

In [10]:
# Fit the model to the training data.
x_train_unroll = unroll_frames(x_train)
y_train_unroll = unroll_frames(y_train)
x_val_unroll = unroll_frames(x_val)
y_val_unroll = unroll_frames(y_val)

In [12]:
# Define modifiable training hyperparameters.
epochs = 10
batch_size = 128

# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=0)

# create model to train
model = unet((64, 64, 4), 3, 8)
model.compile(
    loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(),
)

# fit data to model
history = model.fit(
    x_train_unroll,
    y_train_unroll,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = (x_val_unroll, y_val_unroll),
    callbacks=[early_stopping, reduce_lr],
)

Epoch 1/10


2022-04-16 19:18:01.602455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


227/227 [==============================] - ETA: 0s - loss: 0.9288

2022-04-16 19:18:18.403414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


227/227 [==============================] - 19s 76ms/step - loss: 0.9288 - val_loss: 0.6194 - lr: 0.0010
Epoch 2/10
227/227 [==============================] - 15s 67ms/step - loss: 0.5005 - val_loss: 0.4813 - lr: 0.0010
Epoch 3/10
227/227 [==============================] - 15s 68ms/step - loss: 0.4311 - val_loss: 0.4180 - lr: 0.0010
Epoch 4/10
227/227 [==============================] - 15s 66ms/step - loss: 0.3937 - val_loss: 0.3795 - lr: 0.0010
Epoch 5/10
227/227 [==============================] - 15s 65ms/step - loss: 0.3694 - val_loss: 0.3606 - lr: 0.0010
Epoch 6/10
227/227 [==============================] - 15s 68ms/step - loss: 0.3557 - val_loss: 0.3493 - lr: 0.0010
Epoch 7/10
227/227 [==============================] - 16s 68ms/step - loss: 0.3418 - val_loss: 0.3360 - lr: 0.0010
Epoch 8/10
227/227 [==============================] - 15s 67ms/step - loss: 0.3330 - val_loss: 0.3274 - lr: 0.0010
Epoch 9/10
227/227 [==============================] - 15s 66ms/step - loss: 0.3229 - val_lo

In [13]:
model.save('models/keras/{name}'.format(name=model.name))
# model = keras.models.load_model('models/keras/unet8')

2022-04-16 19:20:37.818271: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/keras/unet8/assets


## Animate the Labels and Predictions

In [14]:
def data_to_input(dataset, boundary, example_i):
    boundary = np.repeat(np.expand_dims(boundary, (0, -1)), 64, 0)
    data = dataset[example_i]
    return np.concatenate((data, boundary), -1)

In [15]:
import numpy as np  
import matplotlib.pyplot as plt  
import matplotlib.animation as animation
import os 
%matplotlib inline

FPS = 24
INTERVAL = 1000.0/FPS
NUM_EXAMPLES = val_dataset.shape[0]
NUM_FRAMES = val_dataset.shape[1]
MODEL_NAME = model.name

for ex in range(NUM_EXAMPLES):
    x = data_to_input(val_dataset, boundary, ex)
    # y = model.predict(x)
    
    # CREATE THE LABEL VIDEO
    fig = plt.figure()
    label_ims = []
    for i in range(NUM_FRAMES - 1):
        plt.axis('off')
        label_im = plt.imshow(np.rot90(x[i+1, :, :, 0]))
        label_ims.append([label_im])
    
    dir = 'videos/{name}/ex_{ex}/'.format(name=MODEL_NAME, ex=ex)
    os.makedirs(dir, exist_ok=True)
    ani = animation.ArtistAnimation(fig, label_ims, interval=INTERVAL, blit=True, repeat_delay=1000)
    ani.save(dir+'water_label_64.mp4'.format(name=MODEL_NAME, ex=ex))

    # CREATE THE MODEL'S VIDEO
    pred_ims = []
    x_i = np.expand_dims(x[0], 0)
    for i in range(NUM_FRAMES - 1):
        y = model.predict(x_i)
        plt.axis('off')
        pred_im = plt.imshow(np.rot90(y[0, :, :, 0]))
        pred_ims.append([pred_im])
        
        x_i = np.concatenate((y, np.expand_dims(boundary, (0, -1))), -1)

    ani = animation.ArtistAnimation(fig, pred_ims, interval=INTERVAL, blit=True, repeat_delay=1000)
    ani.save(dir+'water_pred_64.mp4'.format(name=MODEL_NAME, ex=ex))
    
    plt.close()

2022-04-16 19:23:09.971087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
